In [1]:
# !pip install deap
# uncomment and run the first time as most probably deap library may not be already installed

In [2]:
!pip install deap

Defaulting to user installation because normal site-packages is not writeable
                                              0.0/109.8 kB ? eta -:--:--
     -------                                  20.5/109.8 kB ? eta -:--:--
     ----------                            30.7/109.8 kB 435.7 kB/s eta 0:00:01
     ----------                            30.7/109.8 kB 435.7 kB/s eta 0:00:01
     ----------                            30.7/109.8 kB 435.7 kB/s eta 0:00:01
     ----------                            30.7/109.8 kB 435.7 kB/s eta 0:00:01
     ----------                            30.7/109.8 kB 435.7 kB/s eta 0:00:01
     --------------------                  61.4/109.8 kB 172.4 kB/s eta 0:00:01
     -------------------------------       92.2/109.8 kB 227.6 kB/s eta 0:00:01
     -------------------------------       92.2/109.8 kB 227.6 kB/s eta 0:00:01
     ---------------------------------    102.4/109.8 kB 210.4 kB/s eta 0:00:01
     ------------------------------------ 109.8/109.8 k

In [8]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# -----------------------------------
# STEP 1: Problem Definition
# -----------------------------------
# We need to optimize a delivery route by minimizing total travel distance.
# The warehouse is at index 0, and we must visit all locations exactly once.

# Distance matrix representing the travel cost between locations
distance_matrix = np.array([
    [0, 10, 15, 20, 25, 30],  # Warehouse
    [10, 0, 35, 25, 30, 20],  # Location 1
    [15, 35, 0, 30, 20, 25],  # Location 2
    [20, 25, 30, 0, 15, 10],  # Location 3
    [25, 30, 20, 15, 0, 35],  # Location 4
    [30, 20, 25, 10, 35, 0]   # Location 5
])

num_locations = len(distance_matrix)  # Total locations (including warehouse)
population_size = 10   # Number of candidate solutions in each generation
generations = 100      # Number of iterations
mutation_rate = 0.2    # Probability of mutation

# -----------------------------------
# STEP 2: Genetic Algorithm Setup
# -----------------------------------
# We define a DEAP "FitnessMin" class to represent our objective (minimize distance).
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize distance
creator.create("Individual", list, fitness=creator.FitnessMin)  # A delivery route is a list of locations

toolbox = base.Toolbox()

# Function to generate a valid random route (excluding warehouse)
def create_valid_route():
    """Creates a shuffled list of locations excluding the warehouse (index 0)."""
    route = list(range(1, num_locations))  # Locations 1 to num_locations-1
    random.shuffle(route)  # Shuffle locations randomly
    return creator.Individual(route)  # Return as DEAP Individual

toolbox.register("individual", tools.initIterate, creator.Individual, create_valid_route)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# -----------------------------------
# STEP 3: Fitness Function (Objective Function)
# -----------------------------------
# The fitness function calculates the total travel distance for a given route.
def route_distance(individual):
    """Computes total travel distance for a given route."""
    route = [0] + individual + [0]  # Start and end at the warehouse
    total_distance = sum(distance_matrix[route[i], route[i + 1]] for i in range(len(route) - 1))
    return total_distance,  # Return as a tuple (DEAP requires tuple format)

toolbox.register("evaluate", route_distance)


# -----------------------------------
# TODO: Extend the Lab (Student Task)
# -----------------------------------
# Modify the fitness function to include time constraints.
# Steps:
# 1. Assume each location has a preferred delivery time slot (e.g., [2, 4, 1, 3, 5]).
# 2. Penalize routes that visit locations later than their required time.
# 3. Modify the `route_distance` function to include a penalty.
# 4. Re-run the genetic algorithm and analyze changes in the best route.


# -----------------------------------
# STEP 4: Genetic Algorithm Operators
# -----------------------------------
# Crossover: Uses Partially Mapped Crossover (PMX) to swap sections between two parents
def custom_pmx(ind1, ind2):
    """Applies PMX crossover after converting to 0-based indices, then converts back."""
    ind1[:] = [x - 1 for x in ind1]  # Convert to 0-based indices
    ind2[:] = [x - 1 for x in ind2]

    tools.cxPartialyMatched(ind1, ind2)  # Apply PMX crossover

    ind1[:] = [x + 1 for x in ind1]  # Convert back to 1-based indices
    ind2[:] = [x + 1 for x in ind2]
    return ind1, ind2  # Return modified individuals

toolbox.register("mate", custom_pmx)  # Crossover function
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mutation_rate)  # Mutation: Swap elements
toolbox.register("select", tools.selTournament, tournsize=3)  # Selection: Tournament method

# -----------------------------------
# STEP 5: Genetic Algorithm Execution
# -----------------------------------
def genetic_algorithm():
    """Executes the Genetic Algorithm to find the optimal delivery route."""
    population = toolbox.population(n=population_size)  # Create initial population

    # Run the genetic algorithm for a set number of generations
    for generation in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=mutation_rate)  # Apply crossover & mutation
        population = toolbox.select(offspring, k=len(population))  # Select best individuals for next generation

    # Return the best solution found
    best_route = tools.selBest(population, k=1)[0]  # Get best route
    best_distance = route_distance(best_route)[0]  # Compute total distance
    return best_route, best_distance

# Run the genetic algorithm
best_route, best_distance = genetic_algorithm()

# -----------------------------------
# STEP 6: Print Results
# -----------------------------------
print("Best Delivery Route:", [0] + best_route + [0])  # Add warehouse start/end
print("Minimum Distance:", best_distance)


Best Delivery Route: [0, 1, 3, 4, 2, 5, 0]
Minimum Distance: 125


In [12]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# -----------------------------------
# STEP 1: Problem Definition
# -----------------------------------
# We need to optimize a delivery route by minimizing total travel distance.
# The warehouse is at index 0, and we must visit all locations exactly once.

# Distance matrix representing the travel cost between locations
distance_matrix = np.array([
    [0, 10, 15, 20, 25, 30],  # Warehouse
    [10, 0, 35, 25, 30, 20],  # Location 1
    [15, 35, 0, 30, 20, 25],  # Location 2
    [20, 25, 30, 0, 15, 10],  # Location 3
    [25, 30, 20, 15, 0, 35],  # Location 4
    [30, 20, 25, 10, 35, 0]   # Location 5
])

num_locations = len(distance_matrix)  # Total locations (including warehouse)
population_size = 10   # Number of candidate solutions in each generation
generations = 100      # Number of iterations
mutation_rate = 0.2    # Probability of mutation

# -----------------------------------
# STEP 2: Genetic Algorithm Setup
# -----------------------------------
# We define a DEAP "FitnessMin" class to represent our objective (minimize distance).
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize distance
creator.create("Individual", list, fitness=creator.FitnessMin)  # A delivery route is a list of locations

toolbox = base.Toolbox()

# Function to generate a valid random route (excluding warehouse)
def create_valid_route():
    """Creates a shuffled list of locations excluding the warehouse (index 0)."""
    route = list(range(1, num_locations))  # Locations 1 to num_locations-1
    random.shuffle(route)  # Shuffle locations randomly
    return creator.Individual(route)  # Return as DEAP Individual

toolbox.register("individual", tools.initIterate, creator.Individual, create_valid_route)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# -----------------------------------
# STEP 3: Fitness Function (Objective Function)
# -----------------------------------
# The fitness function calculates the total travel distance for a given route.
# Preferred delivery time slots for each location (excluding the warehouse)
time_windows = [0, 2, 4, 1, 3, 5]  # Warehouse is always at time 0

def route_distance(individual):
    """Computes the total travel distance with time constraint penalties."""
    route = [0] + individual + [0]  # Start and end at the warehouse
    total_distance = 0
    time = 0
    penalty = 0
    
    for i in range(len(route) - 1):
        current_location = route[i]
        next_location = route[i + 1]
        
        # Add travel distance
        total_distance += distance_matrix[current_location, next_location]
        
        # Update time (assuming 1 unit time per move)
        time += 1
        
        # Check if we arrive later than the preferred time slot
        if time > time_windows[next_location]:
            penalty += (time - time_windows[next_location]) * 10  # Penalty cost
        
    return total_distance + penalty,


toolbox.register("evaluate", route_distance)


# -----------------------------------
# TODO: Extend the Lab (Student Task)
# -----------------------------------
# Modify the fitness function to include time constraints.
# Steps:
# 1. Assume each location has a preferred delivery time slot (e.g., [2, 4, 1, 3, 5]).
# 2. Penalize routes that visit locations later than their required time.
# 3. Modify the `route_distance` function to include a penalty.
# 4. Re-run the genetic algorithm and analyze changes in the best route.


# -----------------------------------
# STEP 4: Genetic Algorithm Operators
# -----------------------------------
# Crossover: Uses Partially Mapped Crossover (PMX) to swap sections between two parents
def custom_pmx(ind1, ind2):
    """Applies PMX crossover after converting to 0-based indices, then converts back."""
    ind1[:] = [x - 1 for x in ind1]  # Convert to 0-based indices
    ind2[:] = [x - 1 for x in ind2]

    tools.cxPartialyMatched(ind1, ind2)  # Apply PMX crossover

    ind1[:] = [x + 1 for x in ind1]  # Convert back to 1-based indices
    ind2[:] = [x + 1 for x in ind2]
    return ind1, ind2  # Return modified individuals

toolbox.register("mate", custom_pmx)  # Crossover function
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mutation_rate)  # Mutation: Swap elements
toolbox.register("select", tools.selTournament, tournsize=3)  # Selection: Tournament method

# -----------------------------------
# STEP 5: Genetic Algorithm Execution
# -----------------------------------
def genetic_algorithm():
    """Executes the Genetic Algorithm to find the optimal delivery route."""
    population = toolbox.population(n=population_size)  # Create initial population

    # Run the genetic algorithm for a set number of generations
    for generation in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=mutation_rate)  # Apply crossover & mutation
        population = toolbox.select(offspring, k=len(population))  # Select best individuals for next generation

    # Return the best solution found
    best_route = tools.selBest(population, k=1)[0]  # Get best route
    best_distance = route_distance(best_route)[0]  # Compute total distance
    return best_route, best_distance

# Run the genetic algorithm
best_route, best_distance = genetic_algorithm()

# -----------------------------------
# STEP 6: Print Results
# -----------------------------------
print("Best Delivery Route:", [0] + best_route + [0])  # Add warehouse start/end
print("Minimum Distance:", best_distance)


Best Delivery Route: [0, 1, 3, 5, 2, 4, 0]
Minimum Distance: 205


In [14]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# -----------------------------------
# STEP 1: Problem Definition
# -----------------------------------
# We need to optimize a delivery route by minimizing total travel distance.
# The warehouse is at index 0, and we must visit all locations exactly once.

# Distance matrix representing the travel cost between locations
distance_matrix = np.array([
    [0, 10, 15, 20, 25, 30],  # Warehouse
    [10, 0, 35, 25, 30, 20],  # Location 1
    [15, 35, 0, 30, 20, 25],  # Location 2
    [20, 25, 30, 0, 15, 10],  # Location 3
    [25, 30, 20, 15, 0, 35],  # Location 4
    [30, 20, 25, 10, 35, 0]   # Location 5
])

num_locations = len(distance_matrix)  # Total locations (including warehouse)
population_size = 10   # Number of candidate solutions in each generation
generations = 100      # Number of iterations
mutation_rate = 0.2    # Probability of mutation

# -----------------------------------
# STEP 2: Genetic Algorithm Setup
# -----------------------------------
# We define a DEAP "FitnessMin" class to represent our objective (minimize distance).
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize distance
creator.create("Individual", list, fitness=creator.FitnessMin)  # A delivery route is a list of locations

toolbox = base.Toolbox()

# Function to generate a valid random route (excluding warehouse)
def create_valid_route():
    """Creates a shuffled list of locations excluding the warehouse (index 0)."""
    route = list(range(1, num_locations))  # Locations 1 to num_locations-1
    random.shuffle(route)  # Shuffle locations randomly
    return creator.Individual(route)  # Return as DEAP Individual

toolbox.register("individual", tools.initIterate, creator.Individual, create_valid_route)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# -----------------------------------
# STEP 3: Fitness Function (Objective Function)
# -----------------------------------
# The fitness function calculates the total travel distance for a given route.
# Preferred delivery time slots for each location (excluding the warehouse)
#time_windows = [0, 2, 4, 1, 3, 5]  # Warehouse is always at time 0

# Preferred delivery time slots for each location (excluding warehouse at index 0)
time_windows = [0, 2, 4, 1, 3, 5]  # Warehouse has no penalty (time 0)


# Updated fitness function with time penalties
def route_distance(individual):
    """Computes total travel distance with penalties for late deliveries."""
    route = [0] + individual + [0]  # Start and end at the warehouse
    total_distance = 0
    current_time = 0
    penalty = 0
    
    for i in range(len(route) - 1):
        current_location = route[i]
        next_location = route[i + 1]
        
        # Add travel distance
        total_distance += distance_matrix[current_location, next_location]
        
        # Increment time (assumes each movement takes 1 time unit)
        current_time += 1
        
        # Check for late delivery and apply penalty
        preferred_time = time_windows[next_location]
        if current_time > preferred_time:
            penalty += (current_time - preferred_time) * 10  # Penalty for lateness
        
    return total_distance + penalty,  # Return total cost (distance + penalty)

# Register the updated evaluation function
toolbox.register("evaluate", route_distance)



# -----------------------------------
# TODO: Extend the Lab (Student Task)
# -----------------------------------
# Modify the fitness function to include time constraints.
# Steps:
# 1. Assume each location has a preferred delivery time slot (e.g., [2, 4, 1, 3, 5]).
# 2. Penalize routes that visit locations later than their required time.
# 3. Modify the `route_distance` function to include a penalty.
# 4. Re-run the genetic algorithm and analyze changes in the best route.


# -----------------------------------
# STEP 4: Genetic Algorithm Operators
# -----------------------------------
# Crossover: Uses Partially Mapped Crossover (PMX) to swap sections between two parents
def custom_pmx(ind1, ind2):
    """Applies PMX crossover after converting to 0-based indices, then converts back."""
    ind1[:] = [x - 1 for x in ind1]  # Convert to 0-based indices
    ind2[:] = [x - 1 for x in ind2]

    tools.cxPartialyMatched(ind1, ind2)  # Apply PMX crossover

    ind1[:] = [x + 1 for x in ind1]  # Convert back to 1-based indices
    ind2[:] = [x + 1 for x in ind2]
    return ind1, ind2  # Return modified individuals

toolbox.register("mate", custom_pmx)  # Crossover function
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mutation_rate)  # Mutation: Swap elements
toolbox.register("select", tools.selTournament, tournsize=3)  # Selection: Tournament method

# -----------------------------------
# STEP 5: Genetic Algorithm Execution
# -----------------------------------
def genetic_algorithm():
    """Executes the Genetic Algorithm to find the optimal delivery route."""
    population = toolbox.population(n=population_size)  # Create initial population

    # Run the genetic algorithm for a set number of generations
    for generation in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=mutation_rate)  # Apply crossover & mutation
        population = toolbox.select(offspring, k=len(population))  # Select best individuals for next generation

    # Return the best solution found
    best_route = tools.selBest(population, k=1)[0]  # Get best route
    best_distance = route_distance(best_route)[0]  # Compute total distance
    return best_route, best_distance

# Run the genetic algorithm
#best_route, best_distance = genetic_algorithm()

# Run the modified genetic algorithm
best_route, best_distance = genetic_algorithm()

# -----------------------------------
# STEP 6: Print Results
# -----------------------------------
#print("Best Delivery Route:", [0] + best_route + [0])  # Add warehouse start/end
#print("Minimum Distance:", best_distance)

# Display updated results
print("Updated Best Delivery Route:", [0] + best_route + [0])
print("Minimum Distance (with penalties):", best_distance)

Updated Best Delivery Route: [0, 4, 2, 1, 3, 5, 0]
Minimum Distance (with penalties): 245


In [11]:
# Write the answers to the questions here
1. Before modifying the fitness function, what was the optimal route and distance?

    We will run the original genetic algorithm (without the time penalty) and record:

    Optimal delivery route
    Minimum travel distance

2. After including time constraints, how did the route change? Did the cost increase?

    We will:

    Re-run the algorithm with the penalized fitness function.
    Compare how the route and the total cost (distance + penalty) have changed.

3. What happens when you increase the number of generations? What happens when you update the mutation rate, 
or change the size of the population? Briefly explain the observations. 

    We will experiment by adjusting:

    Generations: Increased to allow more iterations for the algorithm to improve the solution.
    Mutation Rate: Changes how often small random tweaks occur in the route.
    Population Size: Determines how many candidate solutions exist per generation.
